In [134]:
def Simplified_simulation(t_int,t_end,N,tau_x,tau_y,tau_z):
    dt = float(t_end - t_int) / N

    ### Initialize Matrix for all possible cells at all time points for all dimension of data (lets say we're not going to let it get more than 100)
    Intrinsic_Noise = np.empty((N, 3))
    Intrinsic_Noise[:,:]= np.NAN

    Gene = np.empty((N, 3))
    Gene[:,:]= np.NAN


    g_y=.1
    g_z=.1

    lambda_x=(1.0 - np.exp(-dt / tau_x))

    ### initializing values

    cellular_indexes = np.array(range(100))
    # for index in cellular_indexes:
    Gene[0, 0] = 0
    Gene[0, 1] = 0
    Gene[0, 2] = 0

    for t in np.arange(N-1)+1:
        Gene[t,0] = Gene[t - 1,0] + dt * (-Gene[t - 1,0]/tau_x)+np.sqrt(2/tau_x)*np.random.normal(scale=np.sqrt(dt),loc=0.0)
        Gene[t,1] = Gene[t - 1,1] + dt * (g_y*Gene[t - 1,0]-Gene[t - 1,1]/tau_y)+np.sqrt(2/tau_y)*np.random.normal(scale=np.sqrt(dt),loc=0.0)
        Gene[t, 2] = Gene[t - 1,2] + dt * (g_z*Gene[t - 1,0]-Gene[t - 1,2]/tau_z)+np.sqrt(2/tau_z)*np.random.normal(loc=0.0,scale=np.sqrt(dt))
    return Gene

In [216]:
def analytic_xcor_XY(t,tx,ty,gy):
    values=np.zeros((len(t),1)).ravel()
    tg0=t[t>0]
    R_AA=np.sqrt(np.pi/2)*(1/(2*tx))
    R_BB = (gy**2*ty**2*np.sqrt(np.pi/2))*((ty-tx)/(2*(ty**2-tx**2)))# maybe no 2x?0.0
    NAB=1/np.sqrt(R_AA*R_BB)
    values[t>0]=.5*gy*ty*np.sqrt(np.pi/2)*np.exp(-tg0/tx)/(tx+ty)
    tl0 = t[t <= 0]
    #normalization factors
    # values[t <= 0]=gy*ty*np.sqrt(np.pi/2)*(2*ty*np.exp(tl0/tx)//(ty**2-tx**2))
    values[t <= 0] =.5*gy*ty*np.sqrt(np.pi/2)*(((2*ty*np.exp(tl0/ty))/(ty**2-tx**2))-(np.exp(tl0/tx))/(ty-tx))
    return values*NAB


In [210]:
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
fig, ax1 = plt.subplots()

t_end = 200000
t_int = 0
N = 400000
dt = float(t_end - t_int) / N
time_vect= np.arange(t_int, t_end, dt)
tau_x=5.00000002/np.log(2)
tau_y=30/np.log(2)
tau_z=30.00001/np.log(2)


# axes2 = [ax6, ax7, ax8, ax9, ax10]
Gene = Simplified_simulation(t_int,t_end,N,tau_x,tau_y,tau_z)
cc_storage1 = np.zeros((len(Gene[:, 0]) * 2, 1))
cc_storage1[len(Gene[:, 0]):, 0] = sm.tsa.stattools.ccf(Gene[:, 0], Gene[:, 1])
cc_storage1[0:len(Gene[:, 0]), 0] = sm.tsa.stattools.ccf(Gene[:, 1], Gene[:, 0])[::-1]


In [217]:
x_vect=np.linspace(-t_end,t_end,2*len(time_vect))
ax1.scatter(x_vect,cc_storage1,label='X*Y',color='red')
ax1.plot(x_vect,analytic_xcor_XY(x_vect,tau_x,tau_y,0.1),color='red')
ax1.legend()
ax1.set_xlim([-500,500])
ax1.set_ylim([-.2,1.2])
ax1.set_title('Cross Correlation')
ax1.set_xlabel('Tau')

fig.savefig('temp.png')

In [205]:
### creation of xcor calculation dataframe
xcor_df=np.zeros((100,51))

In [206]:
simulation_range=np.linspace(500,50000,100)
index_vals=np.linspace(-500,500,51)
for z,value in enumerate(simulation_range):

    t_end = int(value)
    t_int = 0
    N = int(value*2)
    dt = float(t_end - t_int) / N
    time_vect= np.arange(t_int, t_end, dt)
    Gene = Simplified_simulation(t_int,t_end,N,tau_x,tau_y,tau_z)
    cc_storage1 = np.zeros((len(Gene[:, 0]) * 2, 1))
    cc_storage1[len(Gene[:, 0]):, 0] = sm.tsa.stattools.ccf(Gene[:, 0], Gene[:, 1])
    cc_storage1[0:len(Gene[:, 0]), 0] = sm.tsa.stattools.ccf(Gene[:, 1], Gene[:, 0])[::-1]
    x_vect=np.linspace(-t_end,t_end,2*len(time_vect))
    f = interp1d(x_vect.ravel(), cc_storage1.ravel())
    xcor_df[z,:]=f(index_vals)

In [226]:
from IPython.display import HTML
from matplotlib import animation, rc
fig2,(ax2,ax3) = plt.subplots(1,2)

# ax2.scatter(index_vals,xcor_df[0,:])
ax2.plot(x_vect,analytic_xcor_XY(x_vect,tau_x,tau_y,0.1),linewidth=3,color='red')
ax2.set_xlim([-500,500])
ax2.set_ylim([-.2,1.2])
dots = ax2.plot([],[],color='blue',linestyle='none',marker='o',label='X')
timetext=ax2.text(0.5,50,'')
index_vals=np.linspace(-500,500,51)
def init():
#     data=np.hstack((index_vals[:],xcor_df[0,:]))
    dots[0].set_data(index_vals[:],xcor_df[0,:])


def animate(i):
#     data=np.hstack((index_vals[:],xcor_df[i,:]))
    dots[0].set_data(index_vals[:],xcor_df[i,:])
    timetext.set_text(str(simulation_range[i]))
    return dots

    

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig2, animate,init_func=init,
                               frames=100, interval=500, blit=False)
HTML(anim.to_html5_video())

AttributeError: 'numpy.ndarray' object has no attribute 'plot'